In [1]:
import os
from _src import LOG_DIR, DB_DIR
from _src import search_log_files

In [18]:
from _src import search_log_files

import os
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def get_color_scale():
    return (px.colors.qualitative.Plotly 
            + px.colors.qualitative.D3 
            + px.colors.qualitative.G10 
            + px.colors.qualitative.T10)

# Function to convert hex to rgba
def hex_to_rgba(hex_color, alpha=0.2):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {alpha})'

# Plot and save figure function
def plot_and_save_fig(sampler_files, db_folder, color_scale, save_path, width=800, height=600, title=""):
    fig = go.Figure()

    # Loop over each sampler and assign colors from the Plotly qualitative scale
    for idx, (sampler, db_files) in enumerate(sampler_files.items()):
        dataframes = []
        for db_file in db_files:
            path = os.path.join(db_folder, db_file)
            storage_url = f"sqlite:///{path}"

            study_name = db_file.split('_bo_')[1].replace('.db', '')
            study_name = "bo_" + study_name if "parafac" in sampler else "ackley_bo_" + study_name

            study = optuna.load_study(study_name=study_name, storage=storage_url)

            df = study.trials_dataframe().copy()
            if study.direction == optuna.study.StudyDirection.MAXIMIZE:
                df['best_value'] = df['value'].cummax()
            else:
                df['best_value'] = df['value'].cummin()
            dataframes.append(df[['number', 'best_value']])

        merged_df = pd.concat(dataframes, axis=1, keys=[f'seed_{i}' for i in range(len(db_files))])
        best_values = merged_df.xs('best_value', axis=1, level=1)
        mean_best = best_values.mean(axis=1)
        std_best = best_values.std(axis=1)

        upper_bound = mean_best + std_best
        lower_bound = mean_best - std_best

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=mean_best,
            mode='lines',
            name=f'Mean Best Value ({sampler})',
            line=dict(color=color_scale[idx])
        ))

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=upper_bound,
            mode='lines',
            line=dict(width=0),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=lower_bound,
            mode='lines',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=hex_to_rgba(color_scale[idx]),
            showlegend=False
        ))

    fig.update_layout(
        title=title,
        xaxis_title="Iteration",
        yaxis_title="Best Value",
        template="plotly_white",
        legend=dict(
            orientation="h",
            x=0,
            y=1
        ),
        width=width,
        height=height
    )

    fig.write_image(save_path)

In [11]:
test_func = "ackley"
benchmark_path_base = "/Users/keisukeonoue/ws/constrained_BO_v2/results_benchmark"
benchmark_path = os.path.join(benchmark_path_base, test_func)

benchmark_log_dir = os.path.join(benchmark_path, "logs")
benchmark_db_dir = os.path.join(benchmark_path, "dbs")

problem_setting = {
    "dim": ["dim2", "dim3", "dim5", "dim7"],
    "model": ["random", "tpe", "gp"]
}

log_files = {dim: {} for dim in problem_setting["dim"]}

for dim in problem_setting["dim"]:
    model_files = {model: [] for model in problem_setting["model"] if model != "parafac"}

    for model in problem_setting["model"]:
        if model == "parafac":
            pass
        else:
            db_files = search_log_files(benchmark_db_dir, [dim, model])
            if len(db_files) == 5:
                model_files[model].extend(db_files)
            
    log_files[dim] = model_files

    

display(log_files)


{'dim2': {'random': ['2024-10-25_13-44-50_ackley_bo_benchmark_random_dim2_seed0.db',
   '2024-10-25_13-44-56_ackley_bo_benchmark_random_dim2_seed1.db',
   '2024-10-25_13-45-03_ackley_bo_benchmark_random_dim2_seed2.db',
   '2024-10-25_13-45-09_ackley_bo_benchmark_random_dim2_seed3.db',
   '2024-10-25_13-45-15_ackley_bo_benchmark_random_dim2_seed4.db'],
  'tpe': ['2024-10-25_13-45-21_ackley_bo_benchmark_tpe_dim2_seed0.db',
   '2024-10-25_13-45-30_ackley_bo_benchmark_tpe_dim2_seed1.db',
   '2024-10-25_13-45-41_ackley_bo_benchmark_tpe_dim2_seed2.db',
   '2024-10-25_13-45-51_ackley_bo_benchmark_tpe_dim2_seed3.db',
   '2024-10-25_13-46-00_ackley_bo_benchmark_tpe_dim2_seed4.db'],
  'gp': ['2024-10-25_13-46-11_ackley_bo_benchmark_gp_dim2_seed0.db',
   '2024-10-25_13-46-59_ackley_bo_benchmark_gp_dim2_seed1.db',
   '2024-10-25_13-48-19_ackley_bo_benchmark_gp_dim2_seed2.db',
   '2024-10-25_13-49-11_ackley_bo_benchmark_gp_dim2_seed3.db',
   '2024-10-25_13-50-04_ackley_bo_benchmark_gp_dim2_seed4.db

In [12]:
temp

[[     number    best_value
  0         0  5.422132e+00
  1         1  2.637531e+00
  2         2  2.637531e+00
  3         3  2.637531e+00
  4         4  2.637531e+00
  ..      ...           ...
  495     495  4.440892e-16
  496     496  4.440892e-16
  497     497  4.440892e-16
  498     498  4.440892e-16
  499     499  4.440892e-16
  
  [500 rows x 2 columns],
       number    best_value
  0         0  5.422132e+00
  1         1  5.422132e+00
  2         2  5.422132e+00
  3         3  5.422132e+00
  4         4  2.637531e+00
  ..      ...           ...
  495     495  4.440892e-16
  496     496  4.440892e-16
  497     497  4.440892e-16
  498     498  4.440892e-16
  499     499  4.440892e-16
  
  [500 rows x 2 columns],
       number    best_value
  0         0  1.027576e+01
  1         1  3.625385e+00
  2         2  3.625385e+00
  3         3  3.625385e+00
  4         4  3.625385e+00
  ..      ...           ...
  495     495  4.440892e-16
  496     496  4.440892e-16
  497     497  4.4

In [13]:
test_func = "ackley"
benchmark_path_base = "/Users/keisukeonoue/ws/constrained_BO_v2/results_benchmark"
benchmark_path = os.path.join(benchmark_path_base, test_func)

parafac_path_base = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_8"
parafac_db_dir = os.path.join(parafac_path_base, "dbs")

benchmark_log_dir = os.path.join(benchmark_path, "logs")
benchmark_db_dir = os.path.join(benchmark_path, "dbs")

problem_setting = {
    "dim": ["dim2", "dim3", "dim5", "dim7"],
    "model": ["random", "tpe", "gp", "parafac"]
}

log_files = {dim: {} for dim in problem_setting["dim"]}

for dim in problem_setting["dim"]:
    model_files = {model: [] for model in problem_setting["model"]}

    for model in problem_setting["model"]:
        if model == "parafac":
            # parafac 用の実験結果を取得
            parafac_files = search_log_files(parafac_db_dir, [dim, "parafac"])
            if len(parafac_files) == 5:
                model_files[model].extend(parafac_files)
        else:
            # ベンチマーク用の実験結果を取得
            db_files = search_log_files(benchmark_db_dir, [dim, model])
            if len(db_files) == 5:
                model_files[model].extend(db_files)
            
    log_files[dim] = model_files

# 保存ディレクトリのパス
image_save_base = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5/images"

# log_files の内容に基づいてプロットを生成
for dim, model_files in log_files.items():
    for model, db_files in model_files.items():
        if db_files:  # db_files が空でない場合にのみプロット
            sampler_files = {model: db_files}
            
            # プロットの保存パスを設定
            save_path = os.path.join(image_save_base, f"best_values_plot_{dim}_{model}.png")
            
            # タイトルを設定
            title = f"Ackley of dimension {dim} with model {model}"

            # プロットと保存
            plot_and_save_fig(
                sampler_files=sampler_files,
                db_folder=benchmark_db_dir if model != "parafac" else parafac_db_dir,
                color_scale=get_color_scale(),
                save_path=save_path,
                width=600,
                height=400,
                title=title
            )

print("プロットの保存が完了しました。")



{'dim2': {'random': ['2024-10-25_13-44-50_ackley_bo_benchmark_random_dim2_seed0.db',
   '2024-10-25_13-44-56_ackley_bo_benchmark_random_dim2_seed1.db',
   '2024-10-25_13-45-03_ackley_bo_benchmark_random_dim2_seed2.db',
   '2024-10-25_13-45-09_ackley_bo_benchmark_random_dim2_seed3.db',
   '2024-10-25_13-45-15_ackley_bo_benchmark_random_dim2_seed4.db'],
  'tpe': ['2024-10-25_13-45-21_ackley_bo_benchmark_tpe_dim2_seed0.db',
   '2024-10-25_13-45-30_ackley_bo_benchmark_tpe_dim2_seed1.db',
   '2024-10-25_13-45-41_ackley_bo_benchmark_tpe_dim2_seed2.db',
   '2024-10-25_13-45-51_ackley_bo_benchmark_tpe_dim2_seed3.db',
   '2024-10-25_13-46-00_ackley_bo_benchmark_tpe_dim2_seed4.db'],
  'gp': ['2024-10-25_13-46-11_ackley_bo_benchmark_gp_dim2_seed0.db',
   '2024-10-25_13-46-59_ackley_bo_benchmark_gp_dim2_seed1.db',
   '2024-10-25_13-48-19_ackley_bo_benchmark_gp_dim2_seed2.db',
   '2024-10-25_13-49-11_ackley_bo_benchmark_gp_dim2_seed3.db',
   '2024-10-25_13-50-04_ackley_bo_benchmark_gp_dim2_seed4.db

In [8]:
result_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5"
db_folder = os.path.join(result_path, "dbs")

LOG_DIR = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5/logs"
dir_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5/dbs"


problem_setting = {
    "dim": ["dim3", "dim5", "dim7"],
    "model": ["random", "parafac"]
}

parafac_setting = {
    "rank": ["rank2"],
    "mask": ["mask0.3"],
    "tradeoff": ["tradeoff3.0"]
}

log_files = {dim: {} for dim in problem_setting["dim"]}

for dim in problem_setting["dim"]:
    for rank in parafac_setting["rank"]:
        for tradeoff in parafac_setting["tradeoff"]:
            sampler_files = {sampler: [] for sampler in problem_setting["model"] if sampler != "parafac"}

            for model in problem_setting["model"]:
                if model == "parafac":
                    for mask in parafac_setting["mask"]:
                        parafac_name = f"{model}_rank{rank}_mask{mask}_tradeoff{tradeoff}"
                        sampler_files[parafac_name] = []

                        files = search_log_files(dir_path, [dim, rank, model, mask, tradeoff])
                        if len(files) == 5:
                            sampler_files[parafac_name].extend(files)
                else:
                    files = search_log_files(dir_path, [dim, model])
                    if len(files) == 5:
                        sampler_files[model].extend(files)

            log_files[dim] = sampler_files

            display(log_files)
            print()

            color_scale = px.colors.qualitative.Plotly + px.colors.qualitative.D3 + px.colors.qualitative.G10 + px.colors.qualitative.T10
            save_path = os.path.join(result_path, "images", f"best_values_plot_{dim}_{rank}_{tradeoff}.png")
                
            plot_and_save_fig(
                sampler_files, 
                db_folder, 
                color_scale, 
                save_path,
                width=600,
                height=400,
                title=f'Ackley of dimension {dim}'
            )

{'dim3': {'random': ['2024-10-27_07-42-52_ackley_bo_benchmark_random_dim3_seed0.db',
   '2024-10-27_07-43-34_ackley_bo_benchmark_random_dim3_seed1.db',
   '2024-10-27_07-44-19_ackley_bo_benchmark_random_dim3_seed2.db',
   '2024-10-27_07-45-04_ackley_bo_benchmark_random_dim3_seed3.db',
   '2024-10-27_07-45-46_ackley_bo_benchmark_random_dim3_seed4.db'],
  'parafac_rankrank2_maskmask0.3_tradeofftradeoff3.0': ['2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed0.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed1.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed2.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed3.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed4.db']},
 'dim5': {},
 'dim7': {}}

{'dim3': {'random': ['2024-10-27_07-42-52_ackley_bo_benchmark_random_dim3_seed0.db',
   '2024-10-27_07-43-34_ackley_bo_benchmark_random_dim3_seed1.db',
   '2024-10-27_07-44-19_ackley_bo_benchmark_random_dim3_seed2.db',
   '2024-10-27_07-45-04_ackley_bo_benchmark_random_dim3_seed3.db',
   '2024-10-27_07-45-46_ackley_bo_benchmark_random_dim3_seed4.db'],
  'parafac_rankrank2_maskmask0.3_tradeofftradeoff3.0': ['2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed0.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed1.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed2.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed3.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed4.db']},
 'dim5': {'random': ['2024-10-27_07-46-29_ackley_bo_benchmark_random_dim5_seed0.db',
   '2024-10-27_07-47-18_ackley_bo_benchmark_random_dim5_seed1.db',
   '2024-10-27_07-48-09_ackley_bo_benchmark_rando

{'dim3': {'random': ['2024-10-27_07-42-52_ackley_bo_benchmark_random_dim3_seed0.db',
   '2024-10-27_07-43-34_ackley_bo_benchmark_random_dim3_seed1.db',
   '2024-10-27_07-44-19_ackley_bo_benchmark_random_dim3_seed2.db',
   '2024-10-27_07-45-04_ackley_bo_benchmark_random_dim3_seed3.db',
   '2024-10-27_07-45-46_ackley_bo_benchmark_random_dim3_seed4.db'],
  'parafac_rankrank2_maskmask0.3_tradeofftradeoff3.0': ['2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed0.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed1.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed2.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed3.db',
   '2024-10-26_20-47-15_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed4.db']},
 'dim5': {'random': ['2024-10-27_07-46-29_ackley_bo_benchmark_random_dim5_seed0.db',
   '2024-10-27_07-47-18_ackley_bo_benchmark_random_dim5_seed1.db',
   '2024-10-27_07-48-09_ackley_bo_benchmark_rando

In [4]:
log_files

{'dim3': {}, 'dim5': {}, 'dim7': {}}

In [ ]:
if __name__ == '__main__':
    result_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5"
    db_folder = os.path.join(result_path, "dbs")

    LOG_DIR = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5/logs"
    dir_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5/dbs"


    problem_setting = {
        "dim": ["dim3", "dim5", "dim7"],
        "model": ["random", "parafac"]
    }

    parafac_setting = {
        "rank": ["rank2"],
        "mask": ["mask0.3"],
        "tradeoff": ["tradeoff3.0"]
    }

    log_files = {dim: {} for dim in problem_setting["dim"]}



    for dim in problem_setting["dim"]:
        for rank in parafac_setting["rank"]:
            for tradeoff in parafac_setting["tradeoff"]:
                sampler_files = {sampler: [] for sampler in problem_setting["model"] if sampler != "parafac"}

                for model in problem_setting["model"]:
                    if model == "parafac":
                        for mask in parafac_setting["mask"]:
                            parafac_name = f"{model}_rank{rank}_mask{mask}_tradeoff{tradeoff}"
                            sampler_files[parafac_name] = []

                            files = search_log_files(dir_path, [dim, rank, model, mask, tradeoff])
                            if len(files) == 5:
                                sampler_files[parafac_name].extend(files)
                    else:
                        files = search_log_files(dir_path, [dim, model])
                        if len(files) == 5:
                            sampler_files[model].extend(files)

                log_files[dim] = sampler_files

                color_scale = px.colors.qualitative.Plotly + px.colors.qualitative.D3 + px.colors.qualitative.G10 + px.colors.qualitative.T10
                save_path = os.path.join(result_path, "images", f"best_values_plot_{dim}_{rank}_{tradeoff}.png")
                
                plot_and_save_fig(
                    sampler_files, 
                    db_folder, 
                    color_scale, 
                    save_path,
                    width=600,
                    height=400,
                    title=f'Ackley of dimension {dim}'
                )